In [1]:
import numpy as np
import pandas as pd
from math import ceil, pi, sqrt
import scipy.optimize as sopt

import matplotlib.pyplot as plt

# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'

import plotly.express as px
import plotly.graph_objects as go

### Метод градиентного спуска

In [2]:
def f(X):
    x1, x2 = X[0], X[1]
    return 2*(x1)**2 + 5*(x2)**2 + x1*x2 - 3*x1 - x2 + 5

def grad(X):
    x1, x2 = X[0], X[1]
    grad = np.array([-3+4*x1+x2, -1+x1+10*x2])
    return grad

In [5]:
def gradient_descent(f, grad, X, eps=0.001, alpha=0.2, max_iters=100):
        f_X = f(X)
        gr = grad(X)
        iters = 0
        fs = []
        grads = []
        x1s = []
        x2s = []
        alphs = []        
        while not np.linalg.norm(gr) < eps and iters < max_iters:
            f_X = f(X)
            gr = grad(X)
            iters += 1
            Z = X - alpha * gr
            # print(Z)
            f_Z = f(Z)
            if f_Z < f_X:
                X = Z
                f_X = f_Z
            else:
                alpha /= 2
            fs.append(round(f_X, len(str(eps))))
            grads.append([round(gr[0], 3), round(gr[1], 3)])
            x1s.append(X[0])
            x2s.append(X[1])
            alphs.append(alpha)
        df = pd.DataFrame({'f': fs,
                          'x1': x1s,
                           'x2': x2s,
                          'grad': grads,
                          'alpha': alphs})
        X_opt = X
        f_opt = f(X_opt)
        print(f'Понадобилось {df.shape[0]} итераций')
        return df

In [6]:
X = np.array([-10, 7])
df = gradient_descent(f, grad, X, eps=0.01, alpha=0.01, max_iters=10**6)
df

Понадобилось 216 итераций


,f,x1,x2,grad,alpha
0,357.0173,-9.640000,6.410000,"[-36, 59]",0.01
1,317.5705,-9.288500,5.875400,"[-35.15, 53.46]",0.01
2,283.5746,-8.945714,5.390745,"[-34.279, 48.466]",0.01
3,254.1404,-8.611793,4.951128,"[-33.392, 43.962]",0.01
4,228.5384,-8.286832,4.552133,"[-32.496, 39.899]",0.01
...,...,...,...,...,...
211,3.8718,0.740703,0.026109,"[-0.012, 0.002]",0.01
212,3.8718,0.740814,0.026091,"[-0.011, 0.002]",0.01
213,3.8718,0.740921,0.026074,"[-0.011, 0.002]",0.01
214,3.8718,0.741023,0.026058,"[-0.01, 0.002]",0.01


### Метод Хука-Дживса

In [13]:
def b_search(f, b, h, dim):
    b1 = b
    f1 = f(b1)
    zeros = np.zeros(dim)
    for i in range(dim):
        zeros[i] = 1
        b_tmp = b1 + h * zeros
        f_tmp = f(b_tmp)

        if f_tmp < f1:
            b1 = b_tmp
        elif f_tmp >= f1:
            b_tmp = b1 - h * zeros
            f_tmp = f(b_tmp)
            if f_tmp < f1:
                b1 = b_tmp
        zeros[i] = 0
    return b1
  
def pattern_search(f, X, eps=0.01, max_iters=5000):
    b_prev = X
    dim = b_prev.shape[0]
    h = np.full(dim, eps * 50)
    r = len(str(eps))
    h_eps = np.full(dim, eps)

    b = b_search(f, b_prev, h, dim)
    
    iters = 0   
    while not np.all(h < h_eps) and iters <= max_iters:
        #print(b_prev)
        #print(b)
        iters += 1
        if np.all(b == b_prev):
            h = h/10
            b_prev = b
            b = b_search(f, b, h, dim)
        else:
            b_prev = b
            b = b + 0.1 * (b - b_prev)
            b = b_search(f, b, h, dim)
    #print(h)
    print(f'Минимум функции достигается в точке {np.round(b, r)} и равен {round(f(b), r)}')
    print()
    print(f'Понадобилось итераций: {iters}')
    return np.concatenate((b, [f(b)]))

In [15]:
b = pattern_search(f, X, eps=0.01)

Минимум функции достигается в точке [0.745 0.045] и равен 3.8737

Понадобилось итераций: 28


### Метод наискорейшего спуска

In [7]:
def steepest_descent_method(f, X, eps=0.01, alpha=0.2):
    r = len(str(eps))
    X_k = X.copy()
    def F(alpha_k):
        return f(X_k - alpha_k * grad(X_k))
    iters = 0
    while np.linalg.norm(grad(X_k)) >= eps:
        iters += 1
        res = sopt.minimize(F, 0, bounds=[[0, 5]])
        alpha = res.x
        X_k = X_k - alpha * grad(X_k)
        
    print(f'Минимум функции достигается в точке {np.round(X_k, r)} и равен {round(f(X_k), r)}')
    print()
    print(f'Понадобилось итераций: {iters}')
    
    return X_k[0], X_k[1], f(X_k)

In [8]:
x_stepeest, y_stepeest, z_stepeest =  steepest_descent_method(f, X)

Минимум функции достигается в точке [0.7417 0.0253] и равен 3.8718

Понадобилось итераций: 11


In [10]:
# plot surface
x = np.linspace(-2, 2, 50)
y = x.copy()
xy = np.meshgrid(x, y)
z = f(xy)
 
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z)])

fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True), name='surface')

# gradient descent
x, y, z = df['x1'].tail(1).values[0], df['x2'].tail(1).values[0], df['f'].tail(1).values[0]
fig.add_scatter3d(x=x.flatten(), y=y.flatten(), z=z.flatten(),
                  marker=dict(size=10), name='gradiend_descent')

# pattern_search
x, y, z = b[0], b[1], b[2]
fig.add_scatter3d(x=[x], y=[y], z=[z], 
                  marker=dict(size=10), name='pattern_search')

fig.update_layout(width=700, height=600,
                         title_text='Multidimensional Optimization', legend=dict(
    y=0.99,
    x=0.01
))

# steepest_descent
x, y, z = x_stepeest, y_stepeest, z_stepeest
fig.add_scatter3d(x=[x], y=[y], z=[z], 
                  marker=dict(size=10), name='steepest_descent')

fig.update_layout(width=700, height=600,
                         title_text='Multidimensional Optimization', legend=dict(
    y=0.99,
    x=0.01
))